In [ ]:
import h5py

In [1]:
import tensorflow as tf
import os
import json
import pandas as pd
import re
import numpy as np
import time
import matplotlib.pyplot as plt
import collections
import random
import requests
from math import sqrt
from PIL import Image
from tqdm.auto import tqdm

In [3]:
import os
import pickle
import numpy as np
from tqdm.notebook import tqdm

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add


In [5]:
BASE_DIR = 'images'
WORKING_DIR = 'imag'

In [7]:
def clean_csv(file_path, output_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Handle lines with more columns than expected
    cleaned_lines = []
    for line in lines:
        columns = line.split(',')
        if len(columns) == 10:
            cleaned_lines.append(line)
        # Optionally handle lines with more columns
        elif len(columns) > 10:
            cleaned_lines.append(','.join(columns[:10]) + '\n')

    with open(output_path, 'w') as file:
        file.writelines(cleaned_lines)

clean_csv('imag/styles.csv', 'imag/styles_clean.csv')


In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import collections
import random
import re
import os
from PIL import Image

# Load the styles.csv file with error handling
try:
    styles_df = pd.read_csv('imag/styles.csv', on_bad_lines='warn')
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

# Check if styles_df was defined successfully
if 'styles_df' in locals():
    print(styles_df.head())
else:
    print("Failed to load the DataFrame.")

# Define the label columns
label_columns = ['gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'usage', 'productDisplayName']

# Initialize a mapping dictionary
mapping = {}

# Iterate through the DataFrame rows
for index, row in tqdm(styles_df.iterrows(), total=styles_df.shape[0]):
    image_id = row['id']  # Assuming 'id' is the column name for image ID
    caption = " ".join(str(row[col]) for col in label_columns if col in row and pd.notna(row[col]))
    
    # Clean the image_id if needed (e.g., remove file extension)
    image_id = str(image_id).split('.')[0]  # Adjust based on your image ID format
    
    if image_id not in mapping:
        mapping[image_id] = []
    mapping[image_id].append(caption)

# Convert mapping to DataFrame
captions = pd.DataFrame([(img_id, cap) for img_id, caps in mapping.items() for cap in caps], columns=['image', 'caption'])

# Preprocess captions
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    text = '[start] ' + text + ' [end]'
    return text

captions['caption'] = captions['caption'].apply(preprocess)

# Display a random caption
random_row = captions.sample(1).iloc[0]
print(random_row.caption)

# Construct image paths and verify they exist
image_dir = 'images'  # Set this to the actual path to your image directory
valid_image_paths = []

for img_id in captions['image'].unique():
    img_path = os.path.join(image_dir, f"{img_id}.jpg")  # Adjust extension if needed
    if os.path.exists(img_path):
        valid_image_paths.append(img_path)
    else:
        print(f"Image not found: {img_path}")

print(f"Found {len(valid_image_paths)} valid image paths out of {len(captions['image'].unique())} unique images.")

# Continue with valid_image_paths for further processing


Skipping line 6044: expected 10 fields, saw 11
Skipping line 6569: expected 10 fields, saw 11
Skipping line 7399: expected 10 fields, saw 11
Skipping line 7939: expected 10 fields, saw 11
Skipping line 9026: expected 10 fields, saw 11
Skipping line 10264: expected 10 fields, saw 11
Skipping line 10427: expected 10 fields, saw 11
Skipping line 10905: expected 10 fields, saw 11
Skipping line 11373: expected 10 fields, saw 11
Skipping line 11945: expected 10 fields, saw 11
Skipping line 14112: expected 10 fields, saw 11
Skipping line 14532: expected 10 fields, saw 11
Skipping line 15076: expected 10 fields, saw 12
Skipping line 29906: expected 10 fields, saw 11
Skipping line 31625: expected 10 fields, saw 11
Skipping line 33020: expected 10 fields, saw 11
Skipping line 35748: expected 10 fields, saw 11
Skipping line 35962: expected 10 fields, saw 11
Skipping line 37770: expected 10 fields, saw 11
Skipping line 38105: expected 10 fields, saw 11
Skipping line 38275: expected 10 fields, saw 

      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                             productDisplayName  
0  2011.0  Casual               Turtle Check Men Navy Blue Shirt  
1  2012.0  Casual             Peter England Men Party Blue Jeans  
2  2016.0  Casual                       Titan Women Silver Watch  
3  2011.0  Casual  Manchester United Men Solid Black Track Pants  
4  2012.0  Casual                          Puma Men Grey T-shirt  


100%|██████████████████████████████████████████████████████████████████████████| 44424/44424 [00:05<00:00, 7887.57it/s]


[start] boys apparel bottomwear capris blue summer casual gini and jony boys washed blue 34 length pants [end]
Image not found: images\39403.jpg
Image not found: images\39410.jpg
Image not found: images\39401.jpg
Image not found: images\39425.jpg
Image not found: images\12347.jpg
Found 44419 valid image paths out of 44424 unique images.


In [13]:
# Preprocess captions
def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    text = '[start] ' + text + ' [end]'
    return text

captions['caption'] = captions['caption'].apply(preprocess)

# Display a random caption
random_row = captions.sample(1).iloc[0]
print(random_row.caption)

[start] start women apparel topwear tops multi summer casual tonga women multicoloured top end [end]


In [15]:
# Assuming your images are in .jpg format
image_file_path = f'C:/Users/user/images/{random_row.image}.jpg'  # Add the .jpg extension
im = Image.open(image_file_path)
im.show()


In [17]:
MAX_LENGTH = 40
VOCABULARY_SIZE = 10000
BATCH_SIZE = 32
BUFFER_SIZE = 1000
EMBEDDING_DIM = 512
UNITS = 512

In [19]:
tokenizer = tf.keras.layers.TextVectorization(
    max_tokens=VOCABULARY_SIZE,
    standardize=None,
    output_sequence_length=MAX_LENGTH)

tokenizer.adapt(captions['caption'])

In [21]:
word2idx = tf.keras.layers.StringLookup(
    mask_token="",
    vocabulary=tokenizer.get_vocabulary())

idx2word = tf.keras.layers.StringLookup(
    mask_token="",
    vocabulary=tokenizer.get_vocabulary(),
    invert=True)

In [23]:
import os
import collections
import random

# Define the base directory for your images
image_dir = 'images'  # Adjust this path to where your images are located

# Create a dictionary to map images to captions
img_to_cap_vector = collections.defaultdict(list)
for img, cap in zip(captions['image'], captions['caption']):
    img_to_cap_vector[img].append(cap)

# Shuffle the image keys
img_keys = list(img_to_cap_vector.keys())
random.shuffle(img_keys)

# Split keys into training and validation sets
slice_index = int(len(img_keys) * 0.8)
img_name_train_keys, img_name_val_keys = img_keys[:slice_index], img_keys[slice_index:]

# Create lists for training images and captions with full paths
train_imgs = []
train_captions = []
for imgt in img_name_train_keys:
    # Append the full path for training images
    full_path = os.path.join(image_dir, f"{imgt}.jpg")  # Adjust the extension if needed
    capt_len = len(img_to_cap_vector[imgt])
    train_imgs.extend([full_path] * capt_len)
    train_captions.extend(img_to_cap_vector[imgt])

# Create lists for validation images and captions with full paths
val_imgs = []
val_captions = []
for imgv in img_name_val_keys:
    # Append the full path for validation images
    full_path = os.path.join(image_dir, f"{imgv}.jpg")  # Adjust the extension if needed
    capv_len = len(img_to_cap_vector[imgv])
    val_imgs.extend([full_path] * capv_len)
    val_captions.extend(img_to_cap_vector[imgv])


In [25]:
len(train_imgs), len(train_captions), len(val_imgs), len(val_captions)

(35539, 35539, 8885, 8885)

In [27]:
def load_data(img_path, caption):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [299, 299])  # Use tf.image.resize instead of tf.keras.layers.Resizing
    img = img / 255.0  # Normalize the image
    caption = tokenizer(caption)  # Ensure tokenizer is properly defined
    return img, caption


In [29]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_imgs, train_captions))
train_dataset = train_dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_imgs, val_captions))
val_dataset = val_dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [31]:
# Verify if all image paths exist
import os

def check_image_paths(img_paths):
    missing_paths = [path for path in img_paths if not os.path.exists(path)]
    if missing_paths:
        print(f"Missing paths: {missing_paths}")
    else:
        print("All paths are valid.")

check_image_paths(train_imgs)
check_image_paths(val_imgs)


Missing paths: ['images\\39401.jpg', 'images\\39403.jpg', 'images\\39410.jpg', 'images\\12347.jpg', 'images\\39425.jpg']
All paths are valid.


In [33]:
def load_data(img_path, caption):
    try:
        img = tf.io.read_file(img_path)
        img = tf.io.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [299, 299])  # Use tf.image.resize instead of tf.keras.layers.Resizing
        img = img / 255.0  # Normalize the image
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        img = tf.zeros([299, 299, 3])  # Return a placeholder image if an error occurs
    caption = tokenizer(caption)  # Ensure tokenizer is properly defined
    return img, caption


In [35]:
# Filter out missing image paths
def filter_missing_paths(img_paths, captions):
    existing_paths = [path for path in img_paths if os.path.exists(path)]
    filtered_captions = [caption for path, caption in zip(img_paths, captions) if os.path.exists(path)]
    return existing_paths, filtered_captions

train_imgs, train_captions = filter_missing_paths(train_imgs, train_captions)
val_imgs, val_captions = filter_missing_paths(val_imgs, val_captions)


In [37]:
# Rebuild the datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_imgs, train_captions))
train_dataset = train_dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_imgs, val_captions))
val_dataset = val_dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [39]:
# Verify if all image paths exist
import os

def check_image_paths(img_paths):
    missing_paths = [path for path in img_paths if not os.path.exists(path)]
    if missing_paths:
        print(f"Missing paths: {missing_paths}")
    else:
        print("All paths are valid.")

check_image_paths(train_imgs)
check_image_paths(val_imgs)


All paths are valid.
All paths are valid.


In [41]:
for img_batch, caption_batch in train_dataset.take(1):
    print(f"Image batch shape: {img_batch.shape}")
    print(f"Caption batch shape: {caption_batch.shape}")


Image batch shape: (32, 299, 299, 3)
Caption batch shape: (32, 40)


In [81]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_imgs, train_captions))

train_dataset = train_dataset.map(
    load_data, num_parallel_calls=tf.data.AUTOTUNE
    ).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

val_dataset = tf.data.Dataset.from_tensor_slices(
    (val_imgs, val_captions))

val_dataset = val_dataset.map(
    load_data, num_parallel_calls=tf.data.AUTOTUNE
    ).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [43]:
image_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.2),
        tf.keras.layers.RandomContrast(0.3),
    ]
)

In [45]:
import tensorflow as tf

def CNN_Encoder():
    # Load the InceptionV3 model with custom weights
    inception_v3 = tf.keras.applications.InceptionV3(
        include_top=False,
        weights='inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5',  # Update this path
    )
    inception_v3.trainable = False

    output = inception_v3.output
    output = tf.keras.layers.Reshape((-1, output.shape[-1]))(output)

    cnn_model = tf.keras.models.Model(inception_v3.input, output)
    return cnn_model


In [11]:
def CNN_Encoder():
    inception_v3 = tf.keras.applications.InceptionV3(
        include_top=False,
        weights='imagenet'
    )
    inception_v3.trainable = False

    output = inception_v3.output
    output = tf.keras.layers.Reshape(
        (-1, output.shape[-1]))(output)

    cnn_model = tf.keras.models.Model(inception_v3.input, output)
    return cnn_model

In [13]:
class TransformerEncoderLayer(tf.keras.layers.Layer):

    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.layer_norm_1 = tf.keras.layers.LayerNormalization()
        self.layer_norm_2 = tf.keras.layers.LayerNormalization()
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense = tf.keras.layers.Dense(embed_dim, activation="relu")


    def call(self, x, training):
        x = self.layer_norm_1(x)
        x = self.dense(x)

        attn_output = self.attention(
            query=x,
            value=x,
            key=x,
            attention_mask=None,
            training=training
        )

        x = self.layer_norm_2(x + attn_output)
        return x

In [ ]:
class Embeddings(tf.keras.layers.Layer):

    def __init__(self, vocab_size, embed_dim, max_len):
        super().__init__()
        self.token_embeddings = tf.keras.layers.Embedding(
            vocab_size, embed_dim)
        self.position_embeddings = tf.keras.layers.Embedding(
            max_len, embed_dim, input_shape=(None, max_len))


    def call(self, input_ids):
        length = tf.shape(input_ids)[-1]
        position_ids = tf.range(start=0, limit=length, delta=1)
        position_ids = tf.expand_dims(position_ids, axis=0)

        token_embeddings = self.token_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)

        return token_embeddings + position_embeddings

In [ ]:
Embeddings(tokenizer.vocabulary_size(), EMBEDDING_DIM, MAX_LENGTH)(next(iter(train_dataset))[1]).shape

In [57]:
import os

image_dir = 'images'  # Adjust this path as needed

# List all image files in the directory
image_files = os.listdir(image_dir)

# Print files that are missing or check for specific missing files
missing_files = [f for f in image_files if not os.path.isfile(os.path.join(image_dir, f))]
if missing_files:
    print(f"Missing files: {missing_files}")
else:
    print("All files are present.")


All files are present.


In [59]:
import tensorflow as tf

# Define the file path pattern
file_pattern = 'images/*.jpg'  # Ensure this pattern is correct

# Create dataset
dataset = tf.data.Dataset.list_files(file_pattern)

# Print file paths in the dataset
for file_path in dataset.take(5):  # Check first 5 paths
    print(file_path.numpy().decode('utf-8'))


images\44712.jpg
images\10325.jpg
images\10176.jpg
images\39945.jpg
images\10997.jpg


In [107]:
def parse_image(file_path):
    try:
        image = tf.io.read_file(file_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [224, 224])
        image = image / 255.0
        return image
    except tf.errors.NotFoundError:
        print(f"File not found: {file_path.numpy().decode('utf-8')}")
        return tf.zeros([224, 224, 3])  # Return a default image or handle the error as needed

# Map function over dataset
dataset = tf.data.Dataset.list_files(file_pattern)
dataset = dataset.map(lambda x: parse_image(x))


In [109]:
iterator = iter(dataset)
try:
    for i in range(5):  # Try fetching 5 elements
        image = next(iterator)
        print(f"Image shape: {image.shape}")
except StopIteration:
    print("No more elements in the dataset.")


Image shape: (224, 224, 3)
Image shape: (224, 224, 3)
Image shape: (224, 224, 3)
Image shape: (224, 224, 3)
Image shape: (224, 224, 3)


In [111]:
import tensorflow as tf
import os

# Path to the image directory
image_dir = 'images'
file_pattern = os.path.join(image_dir, '*.jpg')

def parse_image(file_path):
    try:
        image = tf.io.read_file(file_path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [224, 224])
        image = image / 255.0
        return image
    except tf.errors.NotFoundError:
        # Handle file not found error
        print(f"File not found: {file_path.numpy().decode('utf-8')}")
        return tf.zeros([224, 224, 3])  # Return a default image or handle as needed

# Create dataset
dataset = tf.data.Dataset.list_files(file_pattern)

# Map function over dataset
dataset = dataset.map(lambda x: parse_image(x))

# Batch and prefetch for performance
dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# Iterate over dataset
iterator = iter(dataset)
try:
    for i in range(5):  # Try fetching 5 elements
        images = next(iterator)
        print(f"Batch shape: {images.shape}")
except StopIteration:
    print("No more elements in the dataset.")


Batch shape: (32, 224, 224, 3)
Batch shape: (32, 224, 224, 3)
Batch shape: (32, 224, 224, 3)
Batch shape: (32, 224, 224, 3)
Batch shape: (32, 224, 224, 3)


In [113]:
# Define a dataset with limited examples to test
test_dataset = tf.data.Dataset.list_files(file_pattern).take(10)

for file_path in test_dataset:
    try:
        image = parse_image(file_path)
        print(f"Successfully read image: {file_path.numpy().decode('utf-8')}")
    except Exception as e:
        print(f"Error reading image: {file_path.numpy().decode('utf-8')}. Error: {e}")


Successfully read image: images\6386.jpg
Successfully read image: images\29747.jpg
Successfully read image: images\5322.jpg
Successfully read image: images\40786.jpg
Successfully read image: images\4155.jpg
Successfully read image: images\9701.jpg
Successfully read image: images\21138.jpg
Successfully read image: images\9964.jpg
Successfully read image: images\2213.jpg
Successfully read image: images\10396.jpg


In [63]:
class TransformerDecoderLayer(tf.keras.layers.Layer):

    def __init__(self, embed_dim, units, num_heads):
        super().__init__()
        self.embedding = Embeddings(
            tokenizer.vocabulary_size(), embed_dim, MAX_LENGTH)

        self.attention_1 = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.attention_2 = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )

        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()
        self.layernorm_3 = tf.keras.layers.LayerNormalization()

        self.ffn_layer_1 = tf.keras.layers.Dense(units, activation="relu")
        self.ffn_layer_2 = tf.keras.layers.Dense(embed_dim)

        self.out = tf.keras.layers.Dense(tokenizer.vocabulary_size(), activation="softmax")

        self.dropout_1 = tf.keras.layers.Dropout(0.3)
        self.dropout_2 = tf.keras.layers.Dropout(0.5)


    def call(self, input_ids, encoder_output, training, mask=None):
        embeddings = self.embedding(input_ids)

        combined_mask = None
        padding_mask = None

        if mask is not None:
            causal_mask = self.get_causal_attention_mask(embeddings)
            padding_mask = tf.cast(mask[:, :, tf.newaxis], dtype=tf.int32)
            combined_mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.int32)
            combined_mask = tf.minimum(combined_mask, causal_mask)

        attn_output_1 = self.attention_1(
            query=embeddings,
            value=embeddings,
            key=embeddings,
            attention_mask=combined_mask,
            training=training
        )

        out_1 = self.layernorm_1(embeddings + attn_output_1)

        attn_output_2 = self.attention_2(
            query=out_1,
            value=encoder_output,
            key=encoder_output,
            attention_mask=padding_mask,
            training=training
        )

        out_2 = self.layernorm_2(out_1 + attn_output_2)

        ffn_out = self.ffn_layer_1(out_2)
        ffn_out = self.dropout_1(ffn_out, training=training)
        ffn_out = self.ffn_layer_2(ffn_out)

        ffn_out = self.layernorm_3(ffn_out + out_2)
        ffn_out = self.dropout_2(ffn_out, training=training)
        preds = self.out(ffn_out)
        return preds


    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0
        )
        return tf.tile(mask, mult)

In [65]:
class ImageCaptioningModel(tf.keras.Model):

    def __init__(self, cnn_model, encoder, decoder, image_aug=None):
        super().__init__()
        self.cnn_model = cnn_model
        self.encoder = encoder
        self.decoder = decoder
        self.image_aug = image_aug
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")
        self.acc_tracker = tf.keras.metrics.Mean(name="accuracy")


    def calculate_loss(self, y_true, y_pred, mask):
        loss = self.loss(y_true, y_pred)
        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)


    def calculate_accuracy(self, y_true, y_pred, mask):
        accuracy = tf.equal(y_true, tf.argmax(y_pred, axis=2))
        accuracy = tf.math.logical_and(mask, accuracy)
        accuracy = tf.cast(accuracy, dtype=tf.float32)
        mask = tf.cast(mask, dtype=tf.float32)
        return tf.reduce_sum(accuracy) / tf.reduce_sum(mask)


    def compute_loss_and_acc(self, img_embed, captions, training=True):
        encoder_output = self.encoder(img_embed, training=True)
        y_input = captions[:, :-1]
        y_true = captions[:, 1:]
        mask = (y_true != 0)
        y_pred = self.decoder(
            y_input, encoder_output, training=True, mask=mask
        )
        loss = self.calculate_loss(y_true, y_pred, mask)
        acc = self.calculate_accuracy(y_true, y_pred, mask)
        return loss, acc


    def train_step(self, batch):
        imgs, captions = batch

        if self.image_aug:
            imgs = self.image_aug(imgs)

        img_embed = self.cnn_model(imgs)

        with tf.GradientTape() as tape:
            loss, acc = self.compute_loss_and_acc(
                img_embed, captions
            )

        train_vars = (
            self.encoder.trainable_variables + self.decoder.trainable_variables
        )
        grads = tape.gradient(loss, train_vars)
        self.optimizer.apply_gradients(zip(grads, train_vars))
        self.loss_tracker.update_state(loss)
        self.acc_tracker.update_state(acc)

        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}


    def test_step(self, batch):
        imgs, captions = batch

        img_embed = self.cnn_model(imgs)

        loss, acc = self.compute_loss_and_acc(
            img_embed, captions, training=False
        )

        self.loss_tracker.update_state(loss)
        self.acc_tracker.update_state(acc)

        return {"loss": self.loss_tracker.result(), "acc": self.acc_tracker.result()}

    @property
    def metrics(self):
        return [self.loss_tracker, self.acc_tracker]

In [99]:
encoder = TransformerEncoderLayer(EMBEDDING_DIM, 1)
decoder = TransformerDecoderLayer(EMBEDDING_DIM, UNITS, 8)
#import h5py
cnn_model = CNN_Encoder()
caption_model = ImageCaptioningModel(
    cnn_model=cnn_model, encoder=encoder, decoder=decoder, image_aug=image_augmentation,
)

C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [101]:
cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction="none"
)

early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

caption_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=cross_entropy
)

In [143]:
history = caption_model.fit(
    train_dataset,
    epochs=2,
    validation_data=val_dataset,
    callbacks=[early_stopping]
)

Epoch 1/2
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 4122s 4s/step - acc: 0.7756 - loss: 0.8742 - val_acc: 0.7659 - val_loss: 1.0225
Epoch 2/2
1111/1111 ━━━━━━━━━━━━━━━━━━━━ 3962s 4s/step - acc: 0.7806 - loss: 0.8385 - val_acc: 0.7732 - val_loss: 0.9973


In [145]:
idx2word(2).numpy().decode('utf-8')

'end'

In [147]:

def load_image_from_path(img_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.keras.layers.Resizing(299, 299)(img)
    img = tf.cast(img, tf.float32)  # Cast to float32 before division
    img = img / 255.0  # Normalize the image
    return img


def generate_caption(img_path):
    img = load_image_from_path(img_path)
    img = tf.expand_dims(img, axis=0)
    img_embed = caption_model.cnn_model(img)
    img_encoded = caption_model.encoder(img_embed, training=False)

    y_inp = '[start]'
    for i in range(MAX_LENGTH-1):
        tokenized = tokenizer([y_inp])[:, :-1]
        mask = tf.cast(tokenized != 0, tf.int32)
        pred = caption_model.decoder(
            tokenized, img_encoded, training=False, mask=mask)

        pred_idx = np.argmax(pred[0, i, :])
        pred_word = idx2word(pred_idx).numpy().decode('utf-8')
        if pred_word == '[end]':
            break

        y_inp += ' ' + pred_word

    y_inp = y_inp.replace('[start] ', '')
    return y_inp

In [149]:
# Select a random image
idx = random.randrange(0, len(val_imgs))
img_path = val_imgs[idx]

# Generate predicted caption
pred_caption = generate_caption(img_path)

# Retrieve the actual caption
actual_caption = val_captions[idx]  # Make sure this corresponds correctly to val_imgs

# Display the captions and the image
print('Actual Caption:', actual_caption)
print('Predicted Caption:', pred_caption)
print()
Image.open(img_path).show()  #

Actual Caption: [start] start men footwear shoes formal shoes brown summer formal franco leone men brown formal shoes end [end]
Predicted Caption: start men footwear shoes formal shoes brown summer formal franco leone men brown formal shoes end



In [151]:

pred_caption = generate_caption('test_for_fashion/w5.jpg')
print('Predicted Caption:', pred_caption)
print()
#im

Predicted Caption: start women apparel topwear tshirts pink summer casual myntra women pink tshirt end



In [153]:

pred_caption = generate_caption('test_for_fashion/w9.jpg')
print('Predicted Caption:', pred_caption)
print()
#im

Predicted Caption: start men footwear shoes sports shoes white summer sports nike men air impetus white sports shoes end



In [155]:

pred_caption = generate_caption('test_for_fashion/m10.jpg')
print('Predicted Caption:', pred_caption)
print()
#im

Predicted Caption: start men footwear shoes casual shoes brown fall casual rockport mens hydro sail brown blue shoe end



In [159]:

pred_caption = generate_caption('test_for_fashion/w17.jpg')
print('Predicted Caption:', pred_caption)
print()
#im

Predicted Caption: start women accessories jewellery jewellery set gold winter casual estelle women gold jewellery set end



In [161]:

pred_caption = generate_caption('test_for_fashion/wat.jpg')
print('Predicted Caption:', pred_caption)
print()
#im

Predicted Caption: start men accessories watches watches black winter casual casio gshock men black digital watch ex030 end



In [163]:

pred_caption = generate_caption('test_for_fashion/wat.jpg')
print('Predicted Caption:', pred_caption)
print()
#im

Predicted Caption: start men accessories watches watches black winter casual casio gshock men black digital watch ex030 end



In [ ]:
import os
import tensorflow as tf

# Save the model architecture and weights
model_path = 'image_captioning_transformer_model.h5'
weights_path = 'image_captioning_transformer_weights.weights.h5'

# Save the model architecture
caption_model.save(model_path)

# Save the model weights
caption_model.save_weights(weights_path)

# Create a directory for saving files if needed
os.makedirs('Saved Models2', exist_ok=True)

# Move the saved files to the created directory
os.rename(model_path, f'Saved Models2/{model_path}')
os.rename(weights_path, f'Saved Models2/{weights_path}')

# Print paths
print(f'Model saved to: Saved Models2/{model_path}')
print(f'Weights saved to: Saved Models2/{weights_path}')


In [ ]:
import tensorflow as tf

# Define your model architecture
# Make sure to replicate the architecture exactly as it was when you saved it

encoder = TransformerEncoderLayer(EMBEDDING_DIM, 1)
decoder = TransformerDecoderLayer(EMBEDDING_DIM, UNITS, 8)
#import h5py
cnn_model = CNN_Encoder()
caption_model = ImageCaptioningModel(
    cnn_model=cnn_model, encoder=encoder, decoder=decoder, image_aug=image_augmentation,
)
caption_model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=cross_entropy
)
# Load the model weights
caption_model.load_weights('Saved Models2/image_captioning_transformer_weights.weights.h5')

def load_image_from_path(img_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.keras.layers.Resizing(299, 299)(img)
    img = tf.cast(img, tf.float32)  # Cast to float32 before division
    img = img / 255.0  # Normalize the image
    return img


def generate_caption(img_path):
    img = load_image_from_path(img_path)
    img = tf.expand_dims(img, axis=0)
    img_embed = caption_model.cnn_model(img)
    img_encoded = caption_model.encoder(img_embed, training=False)

    y_inp = '[start]'
    for i in range(MAX_LENGTH-1):
        tokenized = tokenizer([y_inp])[:, :-1]
        mask = tf.cast(tokenized != 0, tf.int32)
        pred = caption_model.decoder(
            tokenized, img_encoded, training=False, mask=mask)

        pred_idx = np.argmax(pred[0, i, :])
        pred_word = idx2word(pred_idx).numpy().decode('utf-8')
        if pred_word == '[end]':
            break

        y_inp += ' ' + pred_word

    y_inp = y_inp.replace('[start] ', '')
    return y_inp
# If you saved the complete model, you can load it directly
#caption_model = tf.keras.models.load_model('Saved Models/image_captioning_transformer_model.h5')

# Now you can use the model for predictions
# Example: predictions = caption_model.predict(input_data)


# Example input data (make sure it matches the input shape of your model)
input_data ='test_for_fashion/m11.jpg'# Replace with actual input data


pred_caption = generate_caption('test_for_fashion/m11.jpg')
print('Predicted Caption:', pred_caption)
print()
#im

In [ ]:

pred_caption = generate_caption('test_for_fashion/m11.jpg')
print('Predicted Caption:', pred_caption)
print()
#im